In [14]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto.Random import get_random_bytes
import os

In [15]:
def xor_bytes(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

In [19]:
def aes_128_ofb_encrypt(plaintext, key, iv):
    """
    Encrypt the plaintext using AES-128-OFB mode.

    Args:
        plaintext (bytes): The plaintext to be encrypted.
        key (bytes): The 16-byte (128-bit) encryption key.
        iv (bytes): The 16-byte (128-bit) initialization vector.

    Returns:
        bytes: The encrypted ciphertext.
    """
    # Check if the key and IV are 16 bytes
    if len(key) != 16:
        raise ValueError("Key must be 16 bytes.")
    if len(iv) != 16:
        raise ValueError("IV must be 16 bytes.")
    
    # Check if the plaintext is in bytes
    if type(plaintext) != bytes:
        raise ValueError("Plaintext must be bytes.")

    # Create the AES-128-ECB cipher
    aes_128_ECB = AES.new(key, AES.MODE_ECB)

    # Initialize the OFB state
    state = iv
    
    # Encrypt the plaintext
    ciphertext = b""
    for i in range(0, len(plaintext), 16):
        # Encrypt the state
        state = aes_128_ECB.encrypt(state)

        block = plaintext[i:i+16]

        # padd the block if it is less than 16 bytes
        if len(block) < 16:
            block += b"\x00" * (16 - len(block))

        # XOR the state with the plaintext
        ciphertext += xor_bytes(state, block)

    return ciphertext


In [20]:
def aes_128_ofb_decrypt(ciphertext, key, iv):
    """
    Decrypt the ciphertext using AES-128-OFB mode.

    Args:
        ciphertext (bytes): The ciphertext to be decrypted.
        key (bytes): The 16-byte (128-bit) encryption key.
        iv (bytes): The 16-byte (128-bit) initialization vector.

    Returns:
        bytes: The decrypted plaintext.
    """
    # same as encrypt, just call it with the ciphertext
    plaintext_padded = aes_128_ofb_encrypt(ciphertext, key, iv)
    
    # remove padding
    plaintext = plaintext_padded.rstrip(b"\x00")

    return plaintext

In [35]:
# Example usage
key = get_random_bytes(16)  # Random 128-bit key
iv = get_random_bytes(16)   # Random 128-bit IV

# Plaintext must be at least 512 bits
plaintext = b'This is a 512-bit plaintext for testing AES-128-OFB encryption and decryption.'

# Encrypt
ciphertext = aes_128_ofb_encrypt(plaintext, key, iv)
print("Ciphertext:", ciphertext.hex())

# Decrypt
decrypted = aes_128_ofb_decrypt(ciphertext, key, iv)
print("Decrypted:", decrypted.decode())

Ciphertext: 8cdd3a54d5a49159eb85855fef75492170bf522011332ba9bec638429778370f80f5b20fe69795d819d7e6889a5c7638631586d96aae15fbba7dd2b358e68e64c9d5ad098776dcf9922d968f94b4c2e2
Decrypted: This is a 512-bit plaintext for testing AES-128-OFB encryption and decryption.
